In [9]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

customers = pd.read_csv(r"C:\Users\Anand\Downloads\Customers (1).csv")
products = pd.read_csv(r"C:\Users\Anand\Downloads\Products (1).csv")
transactions = pd.read_csv(r"C:\Users\Anand\Downloads\Transactions.csv")

customer_transactions = transactions.merge(customers, on='CustomerID')
customer_profile = customer_transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum'
}).reset_index()

# Nearest Neighbors model
model = NearestNeighbors(n_neighbors=4)  # 3 lookalikes + 1 for the customer itself
model.fit(customer_profile[['TotalValue', 'Quantity']])

# Finding lookalikes for the first 20 customers
lookalikes = {}
for customer_id in customer_profile['CustomerID'].head(20):
    distances, indices = model.kneighbors(customer_profile[customer_profile['CustomerID'] == customer_id][['TotalValue', 'Quantity']])
    similar_customers = customer_profile.iloc[indices[0]]['CustomerID'].tolist()
    similar_customers.remove(customer_id)  # Remove the customer itself
    lookalikes[customer_id] = similar_customers[:3]  # Top 3 lookalikes

# Save to CSV
lookalike_df = pd.DataFrame(lookalikes.items(), columns=['CustomerID', 'Lookalikes'])
lookalike_df.to_csv('FirstName_LastName_Lookalike.csv', index=False)